In [1]:
!git clone https://github.com/data61/python-paillier

Cloning into 'python-paillier'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 997 (delta 5), reused 0 (delta 0), pack-reused 984
Receiving objects: 100% (997/997), 281.07 KiB | 7.03 MiB/s, done.
Resolving deltas: 100% (637/637), done.


In [2]:
!apt-get install libgmp-dev
!apt-get install libmpfr-dev
!apt-get install libmpc-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgmpxx4ldbl
Suggested packages:
  gmp-doc libgmp10-doc libmpfr-dev
The following NEW packages will be installed:
  libgmp-dev libgmpxx4ldbl
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 325 kB of archives.
After this operation, 1,667 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgmpxx4ldbl amd64 2:6.1.2+dfsg-2 [8,964 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgmp-dev amd64 2:6.1.2+dfsg-2 [316 kB]
Fetched 325 kB in 1s (480 kB/s)
Selecting previously unselected package libgmpxx4ldbl:amd64.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../libgmpxx4ldbl_2%3a6.1.2+dfsg-2_amd64.deb ...
Unpacking libgmpxx4ldbl:amd64 (2:6.1.2+dfsg-2) ...
Selecting previously unselected package libgmp-dev:amd6

In [3]:
!pip3 install -r /content/python-paillier/requirements.txt

     |████████████████████████████████| 450kB 5.5MB/s 
     |████████████████████████████████| 286kB 8.6MB/s 
     |████████████████████████████████| 163kB 8.7MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp36-cp36m-linux_x86_64.whl size=531522 sha256=e3fbca546cd19c6dfae4a0528d623eebae0bc757e3b7d2944f1657ffa367edc9
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
  Created wheel for gmpy2: filename=gmpy2-2.0.8-cp36-cp36m-linux_x86_64.whl size=456719 sha256=b3cfdbc34fad48f8d0dd8adac9a6dd79ad0c7bc6f829cedab7cd270510de385c
  Stored in directory: /root/.cache/pip/wheels/33/5a/45/b4f58933bc61064f49669220932d1d5bcc0b2a1f762f25ac0e
Successfully built pycrypto gmpy2


In [4]:
!python /content/python-paillier/setup.py build

/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:461: UserWarning: Normalizing '1.4.1-dev0' to '1.4.1.dev0'
  warnings.warn(tmpl.format(**locals()))
running build


In [5]:
!python /content/python-paillier/setup.py install

/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:461: UserWarning: Normalizing '1.4.1-dev0' to '1.4.1.dev0'
  warnings.warn(tmpl.format(**locals()))
running install
running bdist_egg
running egg_info
creating phe.egg-info
writing phe.egg-info/PKG-INFO
writing dependency_links to phe.egg-info/dependency_links.txt
writing entry points to phe.egg-info/entry_points.txt
writing requirements to phe.egg-info/requires.txt
writing top-level names to phe.egg-info/top_level.txt
writing manifest file 'phe.egg-info/SOURCES.txt'
reading manifest file 'phe.egg-info/SOURCES.txt'
writing manifest file 'phe.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying phe.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying phe.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying phe.egg-

In [ ]:
#!python /content/python-paillier/setup.py test

In [7]:
%cd /content/python-paillier/

/content/python-paillier


In [64]:
import numpy as np
from sklearn.datasets import load_diabetes

import phe as paillier

seed = 43
np.random.seed(seed)

def get_data(n_clients):
    """
    Import the dataset via sklearn, shuffle and split train/test.
    Return training, target lists for `n_clients` and a holdout test set
    """
    print("Loading data")
    diabetes = load_diabetes()
    y = diabetes.target
    X = diabetes.data
    # Add constant to emulate intercept
    X = np.c_[X, np.ones(X.shape[0])]

    # The features are already preprocessed
    # Shuffle
    perm = np.random.permutation(X.shape[0])
    X, y = X[perm, :], y[perm]

    # Select test at random
    test_size = 50
    test_idx = np.random.choice(X.shape[0], size=test_size, replace=False)
    train_idx = np.ones(X.shape[0], dtype=bool)
    train_idx[test_idx] = False
    X_test, y_test = X[test_idx, :], y[test_idx]
    X_train, y_train = X[train_idx, :], y[train_idx]

    # Split train among multiple clients.
    # The selection is not at random. We simulate the fact that each client
    # sees a potentially very different sample of patients.
    X, y = [], []
    step = int(X_train.shape[0] / n_clients)
    for c in range(n_clients):
        X.append(X_train[step * c: step * (c + 1), :])
        y.append(y_train[step * c: step * (c + 1)])

    return X, y, X_test, y_test


def mean_square_error(y_pred, y):
    """ 1/m * \sum_{i=1..m} (y_pred_i - y_i)^2 """
    return np.mean((y - y_pred) ** 2)


def encrypt_vector(public_key, x):
    return [public_key.encrypt(i) for i in x]


def decrypt_vector(private_key, x):
    return np.array([private_key.decrypt(i) for i in x])


def sum_encrypted_vectors(x, y):
    if len(x) != len(y):
        raise ValueError('Encrypted vectors must have the same size')
    return [x[i] + y[i] for i in range(len(x))]


class Server:
    """Private key holder. Decrypts the average gradient"""

    def __init__(self, key_length):
         keypair = paillier.generate_paillier_keypair(n_length=key_length)
         self.pubkey, self.privkey = keypair

    def decrypt_aggregate(self, input_model, n_clients):
        return decrypt_vector(self.privkey, input_model) / n_clients


class Client:
    """Runs linear regression with local data or by gradient steps,
    where gradient can be passed in.
    Using public key can encrypt locally computed gradients.
    """

    def __init__(self, name, X, y, pubkey):
        self.name = name
        self.pubkey = pubkey
        self.X, self.y = X, y
        self.weights = np.zeros(X.shape[1])
        self.predictionTreshold = 50

    def fit(self, n_iter, eta=0.01):
        """Linear regression for n_iter"""
        for _ in range(n_iter):
            gradient = self.compute_gradient()
            self.gradient_step(gradient, eta)

    def gradient_step(self, gradient, eta=0.01):
        """Update the model with the given gradient"""
        self.weights -= eta * gradient

    def compute_gradient(self):
        """Compute the gradient of the current model using the training set
        """
        delta = self.predict(self.X) - self.y
        return delta.dot(self.X) / len(self.X)

    def predict(self, X):
        """Score test data"""
        return X.dot(self.weights)

    def encrypted_gradient(self, sum_to=None):
        """Compute and encrypt gradient.
        When `sum_to` is given, sum the encrypted gradient to it, assumed
        to be another vector of the same size
        """
        gradient = self.compute_gradient()
        encrypted_gradient = encrypt_vector(self.pubkey, gradient)

        if sum_to is not None:
            return sum_encrypted_vectors(sum_to, encrypted_gradient)
        else:
            return encrypted_gradient

    def analyze_data_privacy(self, X, Y):
      predicted = 0
      notPredicted = 0
      
      # Data that has been predicted correctly
      predictedDataX = []
      predictedDataY = []

      # Checking model's accuracy
      for i in range(len(X)):
        prediction = self.predict(X[i])

        if(prediction >= Y[i] - self.predictionTreshold and prediction <= Y[i] + self.predictionTreshold):
          predicted += 1
          
          predictedDataX.append(X[i])
          predictedDataY.append(Y[i])
        else:
          notPredicted += 1

      print("Predicted: {} Not predicted: {}".format(predicted/(predicted+notPredicted)*100, notPredicted/(predicted+notPredicted)*100))

      # Skaiciuojamas vidutinis maksimalus modelio epsilonas
      increaseValues = np.repeat(0, 10)
      dataIdxArr = np.repeat(1, 10)

      for dataIdx in range(len(predictedDataX)):
        for collumnIdx in range(len(X[0])):
          label = predictedDataY[dataIdx]
          data = predictedDataX[dataIdx]

          y_prediction = self.predict(data)

          if y_prediction >= label - self.predictionTreshold and y_prediction <= label + self.predictionTreshold:
            epsilon = 0.1
            increase = 0

            dataIdxArr[collumnIdx] += 1

            while((y_prediction >= label - self.predictionTreshold and y_prediction <= label + self.predictionTreshold) and increase < 100):
              tempData = data
              increase += epsilon
              tempData[collumnIdx] += increase

              y_prediction = self.predict(tempData)

            increaseValues[collumnIdx] += increase
        
      # Showing output
      for i in range(len(dataIdxArr)):
        print("{}th collumn maximum epsilon: {}".format(i, increaseValues[i]/dataIdxArr[i]))

      s = 0
      for i in range(10):
        s += increaseValues[i]/dataIdxArr[i]

      print("Model maximum epsilon: {}".format(s/10))
      print("Model treshold: {}".format(self.predictionTreshold))

def federated_learning(X, y, X_test, y_test, config):
    n_clients = config['n_clients']
    n_iter = config['n_iter']
    names = ['Hospital {}'.format(i) for i in range(1, n_clients + 1)]

    # Instantiate the server and generate private and public keys
    # NOTE: using smaller keys sizes wouldn't be cryptographically safe
    server = Server(key_length=config['key_length'])

    # Instantiate the clients.
    # Each client gets the public key at creation and its own local dataset
    clients = []
    for i in range(n_clients):
        clients.append(Client(names[i], X[i], y[i], server.pubkey))

    # The federated learning with gradient descent
    print('Running distributed gradient aggregation for {:d} iterations'
          .format(n_iter))
    for i in range(n_iter):

        # Compute gradients, encrypt and aggregate
        encrypt_aggr = clients[0].encrypted_gradient(sum_to=None)
        for c in clients[1:]:
            encrypt_aggr = c.encrypted_gradient(sum_to=encrypt_aggr)

        # Send aggregate to server and decrypt it
        aggr = server.decrypt_aggregate(encrypt_aggr, n_clients)

        # Take gradient steps
        for c in clients:
            c.gradient_step(aggr, config['eta'])

    print('Error (MSE) that each client gets after running the protocol:')
    for c in range(len(clients)):
        y_pred = clients[c].predict(X_test)
        mse = mean_square_error(y_pred, y_test)
        print('{:s}:\t{:.2f}'.format(clients[c].name, mse))

        # Analyze client's data privacy
        clients[c].analyze_data_privacy(X[c], y[c])

def local_learning(X, y, X_test, y_test, config):
    n_clients = config['n_clients']
    names = ['Hospital {}'.format(i) for i in range(1, n_clients + 1)]

    # Instantiate the clients.
    # Each client gets the public key at creation and its own local dataset
    clients = []
    for i in range(n_clients):
        clients.append(Client(names[i], X[i], y[i], None))

    # Each client trains a linear regressor on its own data
    print('Error (MSE) that each client gets on test set by '
          'training only on own local data:')
    for c in range(len(clients)):
        clients[c].fit(config['n_iter'], config['eta'])
        y_pred = clients[c].predict(X_test)
        mse = mean_square_error(y_pred, y_test)

        print('{:s}:\t{:.2f}'.format(clients[c].name, mse))

        # Analyze client's data privacy
        clients[c].analyze_data_privacy(X[c], y[c])

if __name__ == '__main__':
    config = {
        'n_clients': 1,
        'key_length': 1024,
        'n_iter': 2000,
        'eta': 1.0,
    }

    X, y, X_test, y_test = get_data(n_clients=config['n_clients'])

    #local_learning(X, y, X_test, y_test, config)

    federated_learning(X, y, X_test, y_test, config)

Loading data
Running distributed gradient aggregation for 2000 iterations
Error (MSE) that each client gets after running the protocol:
Hospital 1:	2202.75
Predicted: 63.775510204081634 Not predicted: 36.224489795918366
0th collumn maximum epsilon: 2.99203187250996
1th collumn maximum epsilon: 0.0
2th collumn maximum epsilon: 0.0
3th collumn maximum epsilon: 0.0
4th collumn maximum epsilon: 0.0
5th collumn maximum epsilon: 0.0
6th collumn maximum epsilon: 0.0
7th collumn maximum epsilon: 0.0
8th collumn maximum epsilon: 0.0
9th collumn maximum epsilon: 0.0
Model maximum epsilon: 0.29920318725099604
Model treshold: 50
